# Libraries

In [109]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
import scripts.yt_utils as yt_utils
import googleapiclient.discovery
from dotenv import load_dotenv
import pandas as pd
import json
import os
import fs

In [111]:
INTERIM_PROJECT_DIR = fs.open_fs("../../data/interim")
YT_VIDEOS_CSV = INTERIM_PROJECT_DIR.getsyspath("YT_videos.csv")
INTERIM_VIDEO_COMMENTS_DIR = fs.open_fs("../../data/interim/video_comments")
INTERIM_VIDEO_REQUEST_DIR = fs.open_fs("../../data/interim/video_request")

Read enviromental variables

In [112]:
load_dotenv()

YT_API_KEY = os.getenv('YT-API-KEY')

In [113]:
df = pd.read_csv(YT_VIDEOS_CSV)
df.head()

,publishedAt,channelId,title,description,channelTitle,playlistId,position,videoId,videoOwnerChannelTitle,videoOwnerChannelId
0,2024-10-11T01:21:17Z,UCeYSnd34IPe57sM8hkjWhrw,"Theodore Roosevelt, el gringo más gringo - His...","¡Amigos, acompáñenos el día de hoy a conocer l...",Historia para Tontos Podcast,PLlP9CI1vpjD6siEPRL5TK3S4a6V6KDTIe,0,wCm4FNSnDPs,Historia para Tontos Podcast,UCeYSnd34IPe57sM8hkjWhrw
1,2024-09-26T04:28:43Z,UCeYSnd34IPe57sM8hkjWhrw,Guerra de los 30 años / Paz de Westfalia 🗺 - H...,"¡Amigos, acompáñenos el día de hoy terminaremo...",Historia para Tontos Podcast,PLlP9CI1vpjD6siEPRL5TK3S4a6V6KDTIe,1,5Uac-LKCY4Y,Historia para Tontos Podcast,UCeYSnd34IPe57sM8hkjWhrw
2,2024-09-26T03:36:38Z,UCeYSnd34IPe57sM8hkjWhrw,La Guerra de los 30 años 🗺 - Historia para ton...,"¡Amigos, acompáñenos el día de hoy a explorar ...",Historia para Tontos Podcast,PLlP9CI1vpjD6siEPRL5TK3S4a6V6KDTIe,2,kSRLcVrXYSY,Historia para Tontos Podcast,UCeYSnd34IPe57sM8hkjWhrw
3,2024-09-15T06:45:58Z,UCeYSnd34IPe57sM8hkjWhrw,Maravillas del mundo moderno 🏛 ⛩ - Historia p...,"¡Amigos, acompáñenos el día de hoy a explorar ...",Historia para Tontos Podcast,PLlP9CI1vpjD6siEPRL5TK3S4a6V6KDTIe,3,HxMXLWqe9HQ,Historia para Tontos Podcast,UCeYSnd34IPe57sM8hkjWhrw
4,2024-09-03T02:19:12Z,UCeYSnd34IPe57sM8hkjWhrw,Historia del anime 🗾 - ft César Ruelas - His...,"¡Amigos, acompáñenos el día de hoy a explorar ...",Historia para Tontos Podcast,PLlP9CI1vpjD6siEPRL5TK3S4a6V6KDTIe,4,cOxTBBHDN-0,Historia para Tontos Podcast,UCeYSnd34IPe57sM8hkjWhrw


In [114]:
videoId_list = list(df["videoId"].unique())
print(f"Exist {len(videoId_list)} unique videos")

Exist 25 unique videos


## Pull all videos

In [115]:
api_service_name = "youtube"
api_version = "v3"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=YT_API_KEY)


request = youtube.commentThreads().list(
    part='id,snippet,replies',
    videoId=videoId_list[0]
)
response = request.execute()
response

{'kind': 'youtube#commentThreadListResponse',
 'etag': 'EDs7L7uK_eDTQ6bBPIueYJwARK4',
 'pageInfo': {'totalResults': 19, 'resultsPerPage': 20},
 'items': [{'kind': 'youtube#commentThread',
   'etag': 'LY5WGw22qMyplH0Nvg_9npk26qY',
   'id': 'UgyOG5SwEkLlyrlUz0t4AaABAg',
   'snippet': {'channelId': 'UCeYSnd34IPe57sM8hkjWhrw',
    'videoId': 'wCm4FNSnDPs',
    'topLevelComment': {'kind': 'youtube#comment',
     'etag': 'Cg2MLCwfBhawKKG2-Jb_CppW8zs',
     'id': 'UgyOG5SwEkLlyrlUz0t4AaABAg',
     'snippet': {'channelId': 'UCeYSnd34IPe57sM8hkjWhrw',
      'videoId': 'wCm4FNSnDPs',
      'textDisplay': 'Jajaja pobrecito de Iker! No lo dejan ser!! Que malo eres teca!! Buen episodio. Estuvo muy bueno',
      'textOriginal': 'Jajaja pobrecito de Iker! No lo dejan ser!! Que malo eres teca!! Buen episodio. Estuvo muy bueno',
      'authorDisplayName': '@lorenavelazqueztorres9813',
      'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AIdro_lTVeac9TBBd07-AHzhpJtJaLgoqVGs9xKCtVA_W9cscirrK8ltE9feT

In [117]:
api_service_name = "youtube"
api_version = "v3"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=YT_API_KEY)

for video in videoId_list:
    YT_VIDEOS_COMMENTS_CSV = INTERIM_VIDEO_COMMENTS_DIR.getsyspath(f"video_{video}_commets.csv")
    YT_VIDEOS_COMMENTS_JSON = INTERIM_VIDEO_COMMENTS_DIR.getsyspath(f"video_{video}_commets.json")
    REQUEST_JSON = INTERIM_VIDEO_REQUEST_DIR.getsyspath(f"video_{video}_request.json")

    request = youtube.commentThreads().list(
        part='id,snippet,replies',
        videoId=video,
        maxResults = 100
    )
    response = request.execute()

    with open(REQUEST_JSON, 'w', encoding='utf-8') as f:
        json.dump(response, f, ensure_ascii=False, indent=4)